#  Initialization

In [ ]:
# %pip install hdf5storage
# %pip install timm
# %pip install pytorch-optimizer

In [ ]:
from IPython.display import clear_output
# !pip install matplotlib
# !pip install tdqm
# !pip install scipy
# !pip install torchmetrics
# !pip install pandas-profiling
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
clear_output()

In [ ]:
import torch
print(torch.__version__)

In [1]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


import numpy as np
import wandb

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import torch
from torch import nn
from torch.nn import functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchmetrics import MeanSquaredError

import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as T

import scipy.io
seed_everything(42)

# Dataset

## Loading 3D dataset: --only for 3D model - testing not required but possible--

In [238]:
from skullDS import SkullPhaseDS

# if 'trainset' in locals():
#     del trainset
if 'validset' in locals():
    del validset

dataset_path = "E:/PhaseNN_dataset"
# trainset = SkullPhaseDS(dataset_path + "/train")
validset = SkullPhaseDS(dataset_path + "/valid", valid=True)

E:/PhaseNN_dataset/valid


## Loading ray-mapped dataset:

In [2]:
from skullDS import SkullRayDS

if 'trainset_ray' in locals():
    del trainset_ray
if 'validset_ray' in locals():
    del validset_ray

dataset_path = "E:/PhaseNN_dataset"
# trainset_ray = SkullRayDS(dataset_path + "/train", skull_training=True)
validset_ray = SkullRayDS(dataset_path + "/valid", valid=True, skull_training=True)

E:/PhaseNN_dataset/valid


W:\repos\python\PHASE_PREDICTION_NN\skullDS.py:371: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  common_factor = (1 / torch.tensor(1 + alpha_x.tan().clone().detach() ** 2 + alpha_y.tan().clone().detach() ** 2).sqrt())


Map creation requires  12.04631519317627  sec


W:\repos\python\PHASE_PREDICTION_NN\skullDS.py:117: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  focus_['phases'] = torch.tensor(phase)
W:\repos\python\PHASE_PREDICTION_NN\skullDS.py:119: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  focus_['transpose'] = torch.tensor(transpose)


# 3D Model --heavy and unable to be trained with a small dataset we've got--

In [12]:
import model3d

# 3D Model PYPELINE --heavy and unable to be trained with a small dataset we've got--

## LION lr+loss_f check

In [15]:
if 'wandb_logger' in locals():
    del wandb_logger
    wandb.finish()

In [ ]:
import wandb
wandb.login(key=key)

In [ ]:
import pytorch_optimizer.optimizer.ranger21 as Ranger21
import pytorch_optimizer.optimizer.lion as Lion
import pytorch_optimizer.optimizer.sophia as SophiaH

In [39]:
# %pip install tensorboardX
# %pip install tensorboard

# %pip install -U tensorboardX
# %pip install -U tensorboard

In [19]:
import model3d_pypeline
from model3d_pypeline import train_3d_model

from pytorch_lightning.loggers import WandbLogger
import wandb
import pytorch_optimizer.optimizer.lion as Lion
from pytorch_lightning.callbacks import ModelCheckpoint


lion = lambda params, lr: Lion.Lion(
                                    params, 
                                    lr
                                   )
LOSS_FUNCTIONS = [F.mse_loss]

# set sets and loaders
train_loader = DataLoader(trainset, batch_size=3, shuffle=True)
valid_loader = DataLoader(validset, batch_size=1)

# set LRs
LR = [1e-3, 1e-4]

project="PhasesNN - 1 test, 3270 points, B0"
# let's train this shit (B0):
train_3d_model(project, create_model_phase_prediction_b0(),
               train_loader, valid_loader,
               LR, LOSS_FUNCTIONS,
               max_epochs=15, precision="16-mixed")

W:\Python\Lib\site-packages\pytorch_lightning\utilities\parsing.py:196: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type         | Params
---------------------------------------
0 | model | EfficientNet | 5.0 M 
---------

1


Sanity Checking: 0it [00:00, ?it/s]

W:\Python\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Unable to create or open file.


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

KeyError: 'MaterialMatrix'

In [ ]:
trainer.save_checkpoint("BEST_RUN_1/PhasesNN - 1st test, 3150 points, B0/15_epoch.ckpt")

In [ ]:
lightning_model.load_from_checkpoint('PhasesNN - 1st test, 3270 points, B0/PhasesNN - 1st test, 3270 points, B0-epoch=14-valid_l2_loss=0.08-v1.ckpt')

In [ ]:
x,y = dataset[70]

for ind in range(len(dataset)):
    x_it, y_it = dataset[ind]
    if (x_it - x).abs().max() == 0 and ind != 70:
        print(ind)

## LOSS FUNCTION CHECK

In [ ]:
class LogCoshLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.__name__ = 'LogCoshLoss'

    def forward(self, y_t, y_prime_t):
        ey_t = y_t - y_prime_t
        return torch.mean(torch.log(torch.cosh(ey_t + 1e-12)))


class XTanhLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.__name__ = 'XTanhLoss'

    def forward(self, y_t, y_prime_t):
        ey_t = y_t - y_prime_t
        return torch.mean(ey_t * torch.tanh(ey_t))


class XSigmoidLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.__name__ = 'XSigmoidLoss'

    def forward(self, y_t, y_prime_t):
        ey_t = y_t - y_prime_t
        return torch.mean(2 * ey_t / (1 + torch.exp(-ey_t)) - ey_t)


# MODEL EMBEDDER #CURRENTLY IN PROGRESS#

## Artyom's model

### 2D -> 1D

In [4]:
import timm
import torch
import torchvision
from torch import nn
from torch.nn import functional as F

import segmentation_models_pytorch as smp

In [6]:
from embedder import replace_2d_by_1d
from embedder import Encoder1d,     Decoder1d,     AE_model
from embedder import EncoderLinear, DecoderLinear, AE_linear
from embedder import EncoderWCrop1d,               AE_cropper
from ray_collector import Ray_collector, Ray_collector_pass 

### ResNet-18-based AE with "small" latent dim

In [95]:
n_skulls, B, N_RAYS, L = 1, 3, 256*256, 256
dummy_input = torch.randn((n_skulls*N_RAYS, 1, L))

ec = Encoder1d()
dummy_input.shape, ec(dummy_input).shape

(torch.Size([768, 1, 256]), torch.Size([768, 32]))

In [98]:
n_skulls, B, N_RAYS, L = 1, 3, 256*128, 256 # needs more than 22 GB free RAM even on 256*128 rays
dummy_input = torch.randn((n_skulls*N_RAYS, 1, L))

ec = Encoder1d()
dc = Decoder1d()
dummy_input.shape, dc(ec(dummy_input)).shape

(torch.Size([32768, 1, 256]), torch.Size([32768, 1, 256]))

In [101]:
n_skulls, B, N_RAYS, L = 1, 3, 256*128, 256 # needs more than 22 GB free RAM even on 256*128 rays
dummy_input = torch.randn((n_skulls*N_RAYS, 1, L))

ae = AE_model()
dummy_input.shape,ae(dummy_input).shape

(torch.Size([32768, 1, 256]), torch.Size([32768, 1, 256]))

## Further search: #CURRENTLY IN PROGRESS#

### Linear encoder-decoder:

In [8]:
n_skulls, B, N_RAYS, L = 1, 3, 256*256, 256
dummy_input = torch.randn((n_skulls*N_RAYS, 1, L))

ec = EncoderLinear()
ae = AE_linear()
dummy_input.shape, ec(dummy_input).shape, ae(dummy_input).shape

(torch.Size([65536, 1, 256]),
 torch.Size([65536, 32]),
 torch.Size([65536, 1, 256]))

### Cropper encoder (requires no training before main train cycle - the way to go, actually):

In [24]:
n_skulls, N_RAYS, L = 10, 256*256, 256
# dummy_input = torch.randn((n_skulls*N_RAYS, 1, L))
dummy_input = torch.randn((n_skulls, 256, 256, L))

ec = EncoderWCrop1d()
# ae = AE_cropper()

# tic = time.time()
x_ = ec(dummy_input)
# toc = time.time()
# time_cgpt = toc - tic

dummy_input.shape, x_.shape

(torch.Size([10, 256, 256, 256]), torch.Size([655360, 1, 64]))

### Ray collector (skull->rays->*ray map*):

In [35]:
#  input of collector should be:    n_skulls*256**2 x 32
# output of collector should be:    n_skulls        x 32 x 256 x 256:
from ray_collector import Ray_collector, Ray_collector_pass 

n_skulls, N_RAYS, L = 3, 256*256, 256
dummy_input = torch.randn((n_skulls, 256, 256, 256))

ec = EncoderWCrop1d()
rc = Ray_collector(embedder=ec)

rc(dummy_input).shape

torch.Size([3, 64, 256, 256])

### Karhunen-Loeve #CURRENTLY IN PROGRESS#

After some tests KL encoder requires 48 to 64 coefficients to be efficient with the skull data along all the rays.

In [3]:
from KL_transform import KarhunenLoeveTransform

test_transform = torch.load('KL_transform_alot.pt')

In [4]:
skulls_to_fit = 12+1
rays_to_fit = torch.zeros(skulls_to_fit*256**2, 256)

for ind in range(skulls_to_fit):
    x = next(iter(validset_ray))[0]
    rays_to_fit[(ind*256**2):((ind+1)*256**2), :] = x.reshape(256**2, 256)

data = rays_to_fit
signal_ = data[ 256**2 * (skulls_to_fit-1):256**2 * (skulls_to_fit), :].clone().detach()
data    = data[:256**2 * (skulls_to_fit-1), :]

In [6]:
# import KL_transform
from KL_transform import KarhunenLoeveTransform

a = torch.load('KL_test.pt')

In [5]:
from KL_transform import KarhunenLoeveTransform

# a = KL_transform.KarhunenLoeveTransform(component_number=64)
b = KarhunenLoeveTransform(component_number=64)

In [7]:
b.fit(data)
b.return_error = True
b_coefs = b(signal_.cuda())
b_signal_pred_up, b_MAE, b_MSE = b.coefs2series(b_coefs, signal_.cuda())

The idea now is to train a pair of models:
- encoder-compressor with decoder, that will predict the KL coeffs based on the data along the ray.
  It has to have smaller latent dim somewhere in the middle (about 16 to 32).
- decoder for the KL coeffs to be transformed into ray data

Then those models have to become autoencoder. Ideal outcome is the encoder, that after finetuning will be able to compress the ray data into 16 to 32 coeffs.

## Ray-AE training pypeline --not working properly with the KL (that is in progress)--

In [24]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl

class LightningEmbedder(pl.LightningModule):
    def __init__(self, model, lr=1e-3, loss_foo=None, optimizer=None):
        ## optimizer must be either *LAMBDA model_parameters: optimizer(model_parameters, PARAMETERS)* or *None*
        super().__init__()
        self.save_hyperparameters()
        self.model = model
        self.lr = lr
        self.optimizer = optimizer
        if loss_foo != None:
            self.loss_foo = lambda y_hat, y: loss_foo(y_hat, y)
        else:
            self.loss_foo = lambda y_hat, y: F.mse_loss(y_hat, y)

        # self.loss_foo = nn.BCELoss()
        self.loss_out_dict = {}
        
    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        if self.optimizer != None:
            optimizer = self.optimizer(self.parameters(), lr=self.lr)
        return [optimizer]

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        # x, y = x_test, y_test
        y_hat = self.model(x)

        y_hat_gelu = (F.gelu(y_hat)+0.2) / (0.2 + 1)
        x_____gelu = (F.gelu(x    )+0.2) / (0.2 + 1)
        
        # loss = f(l1, l2):
        l1_loss = F.l1_loss(y_hat, x)
        l2_loss = F.mse_loss(y_hat, x)
        loss    = self.loss_foo(y_hat*x/(0.2**2), x*x/(0.2**2)) + self.loss_foo(y_hat*y_hat/(0.2**2), x*y_hat/(0.2**2)) + 0.2*F.l1_loss(y_hat, x)
        # loss    = F.binary_cross_entropy(y_hat_gelu, x_____gelu) + F.mse_loss(y_hat, x) #______________________________________________________________________________________________
        # log losses:
        self.log('train_loss',       loss, prog_bar=True,  on_epoch=True, on_step=True)
        self.log('train_l1_loss', l1_loss, prog_bar=False, on_epoch=True, on_step=True)
        self.log('train_l2_loss', l2_loss, prog_bar=False, on_epoch=True, on_step=True)
        return loss

    def validation_step(self, valid_batch, batch_idx):
        x, y = valid_batch
        y_hat = self.model(x)

        y_hat_gelu = (F.gelu(y_hat)+0.2) / (0.2 + 1)
        x_____gelu = (F.gelu(x    )+0.2) / (0.2 + 1)
        
        # loss = f(l1, l2):
        l1_loss = F.l1_loss(y_hat, x)
        l2_loss = F.mse_loss(y_hat, x)
        loss    = self.loss_foo(y_hat*x/(0.2**2), x*x/(0.2**2)) + self.loss_foo(y_hat*y_hat/(0.2**2), x*y_hat/(0.2**2)) + 0.2*F.l1_loss(y_hat, x)
        # loss    = F.binary_cross_entropy(y_hat_gelu, x_____gelu) + F.mse_loss(y_hat, x) #______________________________________________________________________________________________
        # log losses:
        self.log('valid_loss',       loss, prog_bar=True,  on_epoch=True, on_step=True)
        self.log('valid_l1_loss', l1_loss, prog_bar=False, on_epoch=True, on_step=True)
        self.log('valid_l2_loss', l2_loss, prog_bar=False, on_epoch=True, on_step=True)
        
        return [loss, x, y_hat]

### Training:

In [25]:
from pytorch_lightning.callbacks import ModelCheckpoint

def create_checkpoint_callbacks(project, run_name, save_top_k=3):
    # saves top-K checkpoints based on "valid_l1_loss" metric
    checkpoint_callback_l1 = ModelCheckpoint(
        save_top_k=save_top_k,
        monitor="valid_l1_loss",
        mode="min",
        dirpath= project + "/" + run_name + "/",
        filename=project + "-{epoch:02d}-{valid_l1_loss:.2f}",
    )
    # saves top-K checkpoints based on "valid_l2_loss" metric
    checkpoint_callback_l2 = ModelCheckpoint(
        save_top_k=save_top_k,
        monitor="valid_l2_loss",
        mode="min",
        dirpath= project + "/",
        filename=project + "-{epoch:02d}-{valid_l2_loss:.2f}",
    )
    return [checkpoint_callback_l1, checkpoint_callback_l2]

LOGIN:

In [ ]:
import wandb
wandb.login(key=key)

In [ ]:
if 'wandb_logger' in locals():
    del wandb_logger
    wandb.finish()

In [ ]:
from pytorch_lightning.loggers import WandbLogger
import wandb
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_optimizer.optimizer.lion as Lion
import datetime

lion = lambda params, lr: Lion.Lion(
                                    params,
                                    lr
                                   )
optimizers = [None]#, lion]
optim_name = ['Adam']#, 'Lion']
LOSS_FUNCTIONS = [F.mse_loss]

train_loader = DataLoader(trainset_ray, batch_size=256*32, shuffle=True)
valid_loader = DataLoader(validset_ray, batch_size=256)

# set LRs
LR = [0.001]

project="AE embedder linear"
ind = 0
# let's train this shit (ResNets):
timm_encoder_names = ['resnet18']
nums_featuremaps = [512]
models = []

for ind_name in range(len(timm_encoder_names)):
    for optim_ind in range(len(optimizers)):
        for lr in LR:
            ind += 1
            model = None
            lightning_model = None
        
            # logger setup
            if 'wandb_logger' in locals():
                del wandb_logger
                wandb.finish()
        
            run_name=f'run#{ind:d}, {lr}, {optim_name[optim_ind]}, position_encoding, MSE(yx, xx)+MSE(yy, xy)+L1' # BCELossWL {timm_encoder_names[ind_name]}, 
            wandb_logger = WandbLogger(project=project, name=run_name)
        
            # Lightning model&trainer
            lightning_model = LightningEmbedder(AE_model(position_encoding=True), lr=lr, optimizer=optimizers[optim_ind]) #
            trainer = pl.Trainer(max_epochs=10, logger=wandb_logger, precision=32,
                                 check_val_every_n_epoch=1, log_every_n_steps=1, #max_time=datetime.timedelta(seconds=3600),
                                 callbacks=create_checkpoint_callbacks(project, run_name))
            trainer.fit(lightning_model, train_loader, valid_loader)

# trainer.fit_loop.max_epochs = 50
# trainer.fit(lightning_model, train_loader, valid_loader)

In [36]:
x, _ = next(iter(valid_loader))
y_hat = lightning_model.forward(x)

In [108]:
model = lightning_model.model
y_hat = model(x)

In [ ]:
ind = 42
plt.plot(y_hat[ind, :, :].squeeze())

# Model RAYS->PHASE --current embedders are not good enough--

## Model:

### Embedder summary:

#### Cropper embedder

In [181]:
class EncoderWCrop1d(nn.Module):
    def __init__(
        self,
        in_features=256,
        latent_dim=32,
    ):
        super().__init__()
        self.in_features = in_features
        self.latent_dim = latent_dim
        self.projection = nn.Sequential(
            nn.Conv1d(
                in_channels=2, out_channels=self.latent_dim, kernel_size=(1,), bias=False
            ),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
        )
            
    def forward(self, x):
        x = x.view(-1,1,x.shape[-1])
        threshold = 0.15 
        
        skull_pos = (x > threshold).float().argmax(dim=-1, keepdim=True)
        x_cut = torch.ones((x.shape[0], 1, 2*self.latent_dim)).to(x.device) * skull_pos / 256
        x = torch.cat((x, x[:,:,:]), dim=2)
        x_cut[:, 0, :self.latent_dim]= x[
            torch.arange(x.shape[0]).unsqueeze(1),
            0,
            skull_pos.squeeze(2) + torch.arange(32).to(x.device)
        ]
        # self.projection = self.projection.to(x.device)
        # features = self.projection(x_cut)
        return x_cut #features

#### Linear embedder + linear model 

In [7]:
class linear_skull_embedder(nn.Module):
    def __init__(
        self,
        layer_number=5,
        embedder_out_features=32
    ):
        super().__init__()
        self.ray_embedder = self.generate_layers(layer_number, embedder_out_features)
        self.indx = torch.arange(256).unsqueeze(1).unsqueeze(2).unsqueeze(3)
        self.indy = torch.arange(256).unsqueeze(0).unsqueeze(2).unsqueeze(3)
        
    def generate_layers(self, layer_number, embedder_out_features):
        ray_embedder = nn.Sequential()
        for ind in range(layer_number):
            if ind != layer_number-1:
                ray_embedder.add_module(f"REmb_{ind:n}", torch.nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1))
            else:
                ray_embedder.add_module(f"REmb_{ind:n}", torch.nn.Linear(in_features=256, out_features=embedder_out_features))

        
        return ray_embedder
        
    def forward(self, x):
    # in:  n_skulls x 256 rays x 256 rays x 256 points
    # out: n_skulls x 256 rays x 256 rays x 32 channels -> permute n_skulls x 32 channels x 256 rays x 256 rays
        out = torch.zeros((x.shape[0], 256, 256, 32))
        out[:, :, :, :] = self.ray_embedder(x[:, self.indx, self.indy, :].squeeze())
        return out.permute(0,3,1,2)


class linear_model(nn.Module):
    def __init__(
        self,
        layer_number=5,
        embedder_layer_number=1,
        embedder_out_features=32,
    ):
        super().__init__()
        if layer_number > 10:
            layer_number = 10
            print('More than 10 layers is forbidden. Changed to max.')
        self.embedder = linear_skull_embedder(embedder_layer_number, embedder_out_features)
        self.archivator = self.generate_archivator(embedder_out_features)
        self.model = self.generate_layers(layer_number, embedder_out_features)

    def generate_layers(self, layer_number, embedder_out_features):
        model = nn.Sequential(nn.Flatten())

        featuremap_num = torch.tensor([16**3 / 2, 16**3 / 2, 16**3 / 2, 16**3 / 2, 16**3 / 4, 16**3 / 4, 16**3 / 8, 16**3 / 8, 16**2, 16**2]).int()

        if layer_number > 1:
            model.add_module(f"linear_{0:n}", nn.Linear(in_features=16*64**2, out_features=featuremap_num[0]))
        else:
            model.add_module(f"linear_{0:n}", nn.Linear(in_features=16*64**2, out_features=256))

        for ind in range(layer_number-1):
            if ind != layer_number-2:
                model.add_module(f"linear_{ind+1:n}", nn.Linear(in_features=featuremap_num[ind], out_features=featuremap_num[ind+1]))
            else:
                model.add_module(f"linear_{ind+1:n}", nn.Linear(in_features=featuremap_num[ind], out_features=256))
        
        return model
        
    def generate_archivator(self, embedder_out_features, n_convs=2):
        if n_convs < 2:
            n_convs=2
        archivator = nn.Sequential()
        for ind in range(n_convs-1):
            archivator.add_module(f"conv_{ind:n}", nn.Sequential(
                nn.Conv2d(in_channels=embedder_out_features, out_channels=embedder_out_features, kernel_size=(3,3), stride=(2,2), padding=(1,1)),
                nn.SiLU(),
                nn.BatchNorm2d(embedder_out_features)
            ))
        archivator.add_module(f"conv_{ind+1:n}", nn.Sequential(
            nn.Conv2d(in_channels=embedder_out_features, out_channels=16, kernel_size=(3,3), stride=(2,2), padding=(1,1)),
            nn.SiLU(),
            nn.BatchNorm2d(16)
        ))
        return archivator

    def forward(self, x):
        device = x.device
        x = self.embedder(x).to(device)
        x = self.archivator(x).to(device)
        x = self.model(x).to(device)
        return x

# x, _ = next(iter(valid_loader))
# x = torch.rand((5, 256, 256, 256)).float()

# model = linear_model(layer_number=1)
# model(x)

#### Ray collector

In [237]:
a = torch.zeros(1,256,256,256)
a

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [12]:
#  input of collector should be:    n_skulls*256**2 x component_number
# output of collector should be:    n_skulls        x component_number x 256 x 256:
class Ray_collector(nn.Module):
    def __init__(
        self, 
        embedder=torch.load('KL_transform.pt'),
    ):
        super().__init__()
        self.embedder = embedder
        print(embedder)
    
    def forward(self, x):
        x = self.embedder(x.reshape((-1,256)))
        if x.shape[0] % 256**2 != 0:
            raise RuntimeError('Wrong number of rays came into collector!')
        n_skulls = int(x.shape[0] / 256**2)
        
        skull_data = torch.zeros((n_skulls, 64, 256, 256)).to(x.device)
        for ind in range(n_skulls):
            skull_data[ind, :, :, :] = self.collect(x[(256**2 * ind) : (256**2 * (ind+1)), :])
        
        return skull_data
        
    def collect(self, x):
        # x.shape = (256**2,32); output.shape = (1,32,256,256):
        if x.shape != torch.Size([256**2, 64]):
            print(x.shape)
            raise RuntimeError('Inner Ray_collector error: collector obtained wrong tensor.Size')
        return x.permute((1,0)).unsqueeze(0).reshape((1,64,256,256))

# class Ray_collector_pass(nn.Module):
#     def __init__(
#         self, 
#         embedder=EncoderWCrop1d(),
#     ):
#         super().__init__()
#         self.embedder = embedder
    
#     def forward(self, x):
#         return x

n_skulls, N_RAYS, L = 3, 256*256, 256
dummy_input = torch.randn((n_skulls*N_RAYS, 1, L))

# ec = EncoderWCrop1d()
rc = Ray_collector()

rc(dummy_input.cuda()).shape

KarhunenLoeveTransform()


torch.Size([3, 64, 256, 256])

### NN model:

In [22]:
import timm

## main func for model creation
def model_phase_prediction_2D():
    model = timm.create_model('tf_efficientnet_b2')
    model.classifier = nn.Sequential(nn.Flatten(),
                                     nn.Linear(1408, 256))
    model.conv_stem = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), bias=False),
        nn.BatchNorm2d(128),
        nn.GELU(),
        nn.Conv2d(128,  64, kernel_size=(3, 3), stride=(1, 1), bias=False),
        nn.BatchNorm2d(64),
        nn.GELU(),
        nn.Conv2d( 64,  32, kernel_size=(3, 3), stride=(2, 2), bias=False),
        nn.BatchNorm2d(32),
        nn.GELU(),
    )
    # print(model)
    return model

def model_phase_prediction_2D_b0():
    model = timm.create_model('tf_efficientnet_b0')
    model.classifier = nn.Sequential(nn.Flatten(),
                                     nn.Linear(1280, 256))
    # model.conv_stem = nn.Conv2d(256, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)

    model.conv_stem = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), bias=False),
        nn.BatchNorm2d(128),
        nn.GELU(),
        nn.Conv2d(128, 32, kernel_size=(3, 3), stride=(2, 2), bias=False),
        nn.BatchNorm2d(32),
        nn.GELU(),
    )
    # print(model)
    return model

In [23]:
class SimplePhaseNN(nn.Module):
    # convolutional: N_skulls x 256^2 rays x 256 / 32 points => (B,C,N) N_skulls x 256 / 32 points x 256^2 points
    def __init__(self, in_channels=32, ray_num=256**2):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=(3,3), stride=(2, 2), padding=(1, 1), bias=False),
            nn.GELU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 1, kernel_size=(3,3), stride=(2, 2), padding=(1, 1), bias=False),
            nn.GELU(),
        )
        self.linear = nn.Sequential(
            nn.Flatten(),
            nn.Linear (in_features=int(ray_num / 4**2), out_features=256),
        )

    def forward(self, x):
        x = x.permute((0,3,1,2))
        x = self.encoder(x)
        x = self.linear (x)
        return x

class Simple2PhaseNN(nn.Module):
    # convolutional: N_skulls x 256^2 rays x 256 / 32 points => (B,C,N) N_skulls x 256 / 32 points x 256^2 points
    def __init__(self, in_channels=32, ray_num=256**2):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 1, kernel_size=(1,1), bias=False),
            nn.GELU(),
        )
        self.linear = nn.Sequential(
            nn.Flatten(),
            nn.Linear (in_features=int(ray_num), out_features=256),
        )

    def forward(self, x):
        x = x.permute((0,3,1,2))
        x = self.encoder(x)
        x = self.linear (x)
        return x

# test_model = SimplePhaseNN(256)
# dummy_input = torch.rand((10, 256, 256, 256))
# test_model(dummy_input).shape

def model_phase_prediction_2D_simple(in_channels=32):
    model = SimplePhaseNN(in_channels)
    return model

def model_phase_prediction_2D_simple2(in_channels=32):
    model = Simple2PhaseNN(in_channels)
    return model

### Lightning model:

In [33]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl

class LightningModelWCropper(pl.LightningModule):
    def __init__(self, model, ray_collector=Ray_collector(), lr=1e-6, loss_combination=[0.25,0.75], loss_foo=None, scheduler=None, optimizer=None, folder="validation/", predicting_phase_inds=[0, 1]):
        ## scheduler must be either *LAMBDA optimizer: scheduler(optimizer, PARAMETERS)* or *None*
        ## optimizer must be either *LAMBDA model_parameters: optimizer(model_parameters, PARAMETERS)* or *None*
        super().__init__()
        self.save_hyperparameters()
        self.model = model
        self.ray_collector = ray_collector
        self.lr = lr
        self.optimizer = optimizer
        self.loss_combination = torch.tensor(loss_combination, dtype=torch.float) / torch.norm(torch.tensor(loss_combination, dtype=torch.float))
        if loss_foo != None:
            self.loss_foo = lambda y_hat, y: loss_foo(y_hat, y)
        else:
            self.loss_foo = lambda y_hat, y: self.phase_l1(y_hat, y)*self.loss_combination[0] + self.phase_l2(y_hat, y)*self.loss_combination[1]

        self.scheduler = scheduler
        self.loss_out_dict = {}
        self.folder = folder
        if len(self.folder) > 0:
            if self.folder[-1] != '/':
                self.folder = self.folder + '/'

        self.predicting_phase_inds = predicting_phase_inds
        
    def forward(self, x):
        return self.model(self.ray_collector(x))

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        if self.optimizer != None:
            optimizer = self.optimizer(self.parameters(), lr=self.lr)
        return [optimizer]

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        x = self.ray_collector(x)#.to(self.device)
        
        y_hat = self.model(x)
        print(y.shape, y_hat.shape)
        
        y     = y    [self.predicting_phase_inds]
        y_hat = y_hat[self.predicting_phase_inds]
        
        # loss = f(l1, l2):
        l1_loss = self.phase_l1(y_hat, y / (torch.pi)-1)
        l2_loss = self.phase_l2(y_hat, y / (torch.pi)-1)
        loss    = self.loss_foo(y_hat, y / (torch.pi)-1)
        # log losses:
        self.log('train_loss',       loss, prog_bar=True,  on_epoch=True, on_step=True)
        self.log('train_l1_loss', l1_loss, prog_bar=False, on_epoch=True, on_step=True)
        self.log('train_l2_loss', l2_loss, prog_bar=False, on_epoch=True, on_step=True)
        return loss

    def validation_step(self, valid_batch, batch_idx):
        x, y = valid_batch
        x = self.ray_collector(x).to(self.device)
        
        y_hat = self.model(x)
        print(y.shape, y_hat.shape)
        
        y     = y    [self.predicting_phase_inds]
        y_hat = y_hat[self.predicting_phase_inds]
        
        # loss = f(l1, l2):
        l1_loss = self.phase_l1(y_hat, y / (torch.pi)-1)
        l2_loss = self.phase_l2(y_hat, y / (torch.pi)-1)
        loss    = self.loss_foo(y_hat, y / (torch.pi)-1)
        # log losses:
        self.log('valid_loss',       loss, prog_bar=True,  on_epoch=True, on_step=True)
        self.log('valid_l1_loss', l1_loss, prog_bar=False, on_epoch=True, on_step=True)
        self.log('valid_l2_loss', l2_loss, prog_bar=False, on_epoch=True, on_step=True)
        self.loss_out_dict[loss] = {'x': x.detach().cpu().to(torch.float32).numpy(), 'pred': y_hat.detach().cpu().to(torch.float32).numpy()}
        
        return [loss, x, y_hat]

    # def validation_step_end(self, validation_step_outputs):
    #     loss, x, y_hat = validation_step_outputs
    #     self.loss_out_dict[loss] = {'x': x.detach().cpu().numpy(), 'pred': y_hat.detach().cpu().numpy()}

    def on_validation_epoch_end(self):
        try:
            max_loss = max(self.loss_out_dict.keys())
            min_loss = min(self.loss_out_dict.keys())
    
            out = {}
            out['max_loss'] = self.loss_out_dict[max_loss]
            out['min_loss'] = self.loss_out_dict[min_loss]
            hdf5storage.savemat(file_name=self.folder + f"validation_outputs_epoch_{self.current_epoch:02d}.mat", 
                                mdict=out, 
                                format='7.3')
        except Exception as e:
            print(e)
        self.loss_out_dict = {}
        
    @staticmethod
    def phase_l2(y, y_hat):
        return F.mse_loss(y, y_hat)
    @staticmethod
    def phase_l1(y, y_hat):
        return F.l1_loss(y, y_hat)

KarhunenLoeveTransform()


## Training:

### Preps:

In [19]:
from pytorch_lightning.callbacks import ModelCheckpoint

def create_checkpoint_callbacks(project, run_name, save_top_k=3):
    # saves top-K checkpoints based on "valid_l1_loss" metric
    checkpoint_callback_l1 = ModelCheckpoint(
        save_top_k=save_top_k,
        monitor="valid_l1_loss",
        mode="min",
        dirpath= project + "/" + run_name + "/",
        filename=project + "-{epoch:02d}-{valid_l1_loss:.2f}",
    )
    # saves top-K checkpoints based on "valid_l2_loss" metric
    checkpoint_callback_l2 = ModelCheckpoint(
        save_top_k=save_top_k,
        monitor="valid_l2_loss",
        mode="min",
        dirpath= project + "/",
        filename=project + "-{epoch:02d}-{valid_l2_loss:.2f}",
    )
    return [checkpoint_callback_l1, checkpoint_callback_l2]

In [ ]:
import wandb
wandb.login(key=key)

### Training itself:

In [193]:
if 'wandb_logger' in locals():
    del wandb_logger
    wandb.finish()

In [34]:
from pytorch_lightning.loggers import WandbLogger
import wandb
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_optimizer.optimizer.lion as Lion
import pytorch_optimizer.optimizer.ranger21 as Ranger21
import pytorch_optimizer.optimizer.sophia as SophiaH
import datetime
from pytorch_lightning.profilers.profiler import Profiler

lion = lambda params, lr: Lion.Lion(
                                    params,
                                    lr
                                   )
ranger = lambda params, lr: Ranger21.Ranger21(
                                              params,
                                              num_iterations=70*50,
                                              lr=lr,
                                              weight_decay=1e-6,
                                             )

optimizers = [None]#, lion]
optim_name = ['Adam']#, 'Lion']
LOSS_FUNCTIONS = [F.mse_loss]

trainset_ray.shuffle = True
train_loader = DataLoader(trainset_ray, batch_size=3, shuffle=False) # shuffle realization exists inside dataset - we need all rays to be from one skull
valid_loader = DataLoader(validset_ray, batch_size=1)


# set LRs
LR = [1e-2]

project="b0 + ray KL-transform"
ind = 0
# let's train this shit:
for lr in LR:
    ind += 1
    model = None
    lightning_model = None

    # logger setup
    if 'wandb_logger' in locals():
        del wandb_logger
        wandb.finish()

    run_name=f'run#{ind:d}, {lr}, lion'
    wandb_logger = WandbLogger(project=project, name=run_name)

    # Lightning model&trainer
    lightning_model = LightningModelWCropper(model_phase_prediction_2D_b0(), 
                                             ray_collector=Ray_collector(embedder=torch.load('KL_transform.pt')), 
                                             lr=lr, optimizer=lion, folder="validation/"+project+'/'+run_name+'/') # 
    trainer = pl.Trainer(max_epochs=15, logger=wandb_logger, precision="16-mixed",
                         check_val_every_n_epoch=1, log_every_n_steps=10,
                         callbacks=create_checkpoint_callbacks(project, run_name))
    trainer.fit(lightning_model, train_loader, valid_loader)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


KarhunenLoeveTransform()


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [46]:
MODEL = LightningModelWCropper(model_phase_prediction_2D(), ray_collector=Ray_collector(), lr=lr, optimizer=ranger, folder="validation/"+project+'/'+run_name+'/')
MODEL.model = lightning_model.model